In [56]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.naukri.com/python-developer-jobs")

wait = WebDriverWait(driver, 4)

# get all job links on the first page
job_links = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.title")))

data = []

for i in range(10):   # sirf 2 jobs test ke liye
    job_links = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.title")))
    
    driver.execute_script("arguments[0].scrollIntoView(true);", job_links[i])
    driver.execute_script("arguments[0].click();", job_links[i])  # JS click

    wait.until(EC.number_of_windows_to_be(2))
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(2)  # wait for page to load
    # ✅ ab BeautifulSoup se sari fields nikalenge
    html = driver.page_source
    page = BeautifulSoup(html, "html.parser")

    # Title
    title_elem = page.find("h1", class_="styles_jd-header-title__rZwM1")
    title = title_elem.get_text(strip=True) if title_elem else "NA"

    # Company
    comp_elem = page.find("a", href=lambda x: x and "jobs-careers" in x)
    company = comp_elem.get_text(strip=True) if comp_elem else "NA"

    # Experience
    exp_elem = page.find("div", class_="styles_jhc__exp__k_giM")
    exp = exp_elem.find("span").get_text(strip=True) if exp_elem else "NA"

    # Salary
    salary_elem = page.find("div", class_="styles_jhc__salary__jdfEC")
    salary = salary_elem.find("span").get_text(strip=True) if salary_elem else "NA"

    # Location
    loc_elem = page.find("span", class_="styles_jhc__location__W_pVs")
    location = loc_elem.find("a").get_text(strip=True) if loc_elem else "NA"


    # Other Details (Role, Industry, Department, etc.)
    details = {}
    details_section = page.find("div", class_="styles_other-details__oEN4O")

    if details_section:
        detail_divs = details_section.find_all("div", class_="styles_details__Y424J")
        for div in detail_divs:
            label = div.find("label").get_text(strip=True).replace(":", "")  # e.g. Role, Industry Type
            value = div.find("span").get_text(" ", strip=True) if div.find("span") else "NA"
            details[label] = value
    else:
        details = {}

    try:
        education = job.find("div", class_="styles_education__KXFkO") \
                    .find("div", class_="styles_details__Y424J").text.strip()
    except:
        education = "Not Available"


    skills_section = page.find("div", class_="styles_key-skill__GIPn_")

    star_skills = []
    normal_skills = []

    if skills_section:
        for a in skills_section.find_all("a", class_="styles_chip__7YCfG"):
            span = a.find("span")
            skill_name = span.get_text(strip=True) if span else "NA"
            
            if a.find("i", class_="ni-icon-jd-save"):
                star_skills.append(skill_name)  # ⭐ preferred skill
            else:
                normal_skills.append(skill_name)  # normal skill
    skills = [star_skills, normal_skills]


    # Job Description
    # page = BeautifulSoup(driver.page_source, "html.parser")   # already define kiya hai

    jd_elem = page.find("div", class_="styles_JDC__dang-inner-html__h0K4t")
    if jd_elem:
        job_description = jd_elem.get_text(" ", strip=True)   # clean text with spaces
    else:
        job_description = "NA"



    data.append({
        "Title": title,
        "Company": company,
        "Experience": exp,
        "Salary": salary,
        "Location": location,
        "Education": education,
        "Skills": skills,
        **details,
        "Description": job_description
    })

    driver.close()
    driver.switch_to.window(driver.window_handles[0])

df = pd.DataFrame(data)
df.to_csv("naukri_jobs.csv", index=False, encoding="utf-8")

driver.quit()
print("✅ Scraping Done! Data saved to naukri_jobs.csv")


✅ Scraping Done! Data saved to naukri_jobs.csv


In [57]:
df.head(11)

,Title,Company,Experience,Salary,Location,Education,Skills,Role,Industry Type,Department,Employment Type,Role Category,Description
0,Python Developer,Trackmind Solutions,0 years,Not Disclosed,Hyderabad,Not Available,"[[Python], [Django, Pandas, Numpy, SQL]]","Back End Developer ,","IT Services & Consulting ,","Engineering - Software & QA ,","Full Time, Permanent",Software Development,Role & responsibilities Preferred candidate pr...
1,"Python Developer ( Bengaluru, Hyderabad, Pune)",Infosys,3 - 8 years,Not Disclosed,Chandigarh,Not Available,"[[Python], [Django, Flask]]","Software Development - Other ,","IT Services & Consulting ,","Engineering - Software & QA ,","Full Time, Permanent",Software Development,"Responsibilities Write clean, maintainable, an..."
2,Python Software Developer & Infosys- Pan India,Infosys,3 - 8 years,Not Disclosed,Pune,Not Available,"[[Python], [Django, Python Development, Flask]]","Software Development - Other ,","IT Services & Consulting ,","Engineering - Software & QA ,","Full Time, Permanent",Software Development,Responsibilities A day in the life of an Infos...
3,Python Software Developer & Infosys- Pan India(F),Infosys,3 - 8 years,Not Disclosed,Pune,Not Available,"[[Python], [Django, Python Development, Flask]]","Software Development - Other ,","IT Services & Consulting ,","Engineering - Software & QA ,","Full Time, Permanent",Software Development,Responsibilities A day in the life of an Infos...
4,Python Developer,Capgemini,4 - 6 years,Not Disclosed,Bengaluru,Not Available,"[[python, python development, javascript, plm,...","Software Development - Other ,","IT Services & Consulting ,","Engineering - Software & QA ,","Full Time, Permanent",Software Development,This role involves the development and applica...
5,python developer senior software,Xerox,8 - 12 years,15-20 Lacs P.A.,Kochi,Not Available,"[[Python developer], [development and implemen...","Software Development - Other ,","IT Services & Consulting ,","Engineering - Software & QA ,","Full Time, Permanent",Software Development,"Purpose : • Responsible for those functions, ..."
6,Python Developer,Global Information Systems Technology (G...,0 - 5 years,4.25-9.25 Lacs P.A.,Hyderabad,Not Available,"[[Development Life Cycle, Javascript, MongoDB]...","Software Development - Other ,","IT Services & Consulting ,","Engineering - Software & QA ,","Full Time, Permanent",Software Development,Role & responsibilities: Outline the day-to-da...
7,Python Software Developer,HCLTech,5 - 9 years,Not Disclosed,Lucknow,Not Available,"[[Python Development], [fast api, Django Rest ...","Software Development - Other ,","IT Services & Consulting ,","Engineering - Software & QA ,","Full Time, Permanent",Software Development,"Mandatory Skills: Python, Microservices, Cloud..."
8,Python developer,LDT Technology,0 - 4 years,Not Disclosed,Zirakpur,Not Available,"[[Django, Python, Flask], []]","Other ,","IT Services & Consulting ,","Other ,","Full Time, Permanent",Other,We are urgently looking for experienced Backen...
9,Python Developer,Wipro,2 - 6 years,Not Disclosed,Hyderabad,Not Available,"[[sql, python, python development, django fram...","Full Stack Developer ,","IT Services & Consulting ,","Engineering - Software & QA ,","Full Time, Permanent",Software Development,Role Purpose The purpose of this role is to de...
